# LLM のメモリ機能とバンディット機能とワークフロー機能の試験実装

(Version: 0.0.2.2)

最近 LLM についてコンテクストエンジニアリングという言葉をよく聞く。実は Web でなく API から使う LLM は「ステートレス」で、会話などの記憶はデフォルトではしてくれない。会話を続けようと思ったら、メッセージをユーザー側で記録しておき、それをいちいち API に与えないといけないのだ。古い記憶を呼び戻したりするには、単純に記録するだけでなく検索基盤も必要となる。そのような基盤をひっくるめてコンテクストと一般的に呼んでおり、それをどううまく構築するかが問題となっている。

特にコンテクストのうち、記憶の基盤は単に「メモリ」と呼ばれている。PC のメモリなどとごっちゃになって、わかりにくくもあるのだが、そう呼ばれているのだからしかたがない。

メモリ機能をなかなか使ってくれないのに業を煮やして、「バンディットに特定のツールを登録するツールみたいなのを用意して、どういうツールを強制して数を増やして欲しいかを AI さん自身が決められる」というバンディット機能を作って、メモリを使うのを強制してみたのが前回。今回はそれを拡張して、ワークフローも実行させられるようにしてみた。

そのアイデアの軌跡は↓にある。

\[cocolog:95619779\](2025年9月)  
《「LLM のメモリ機能を強制的に使うバンディットマシンの試験実装」と「LLM のメモリ機能とバンディット機能の試験実装」を行った。後者がメインの成果物で、メモリ機能の使用増加をどう強制するかから拡張したフレームワーク。 - JRF のひとこと》  
http://jrf.cocolog-nifty.com/statuses/2025/09/post-8225e2.html

基本的にはこれまでの熊剣迷路問題を使い、メモリ機能とバンディット機能と今回新しいワークフロー機能を使う。

簡単な迷路問題ではあるが、かなり本格的なメモリ機能・バンディット機能・ワークフロー機能を用意している。

セマンティックサーチとかを用意するのは、めんどくさかったので、そういうバックエンドはこれまた AI さんに「データベース偽装」をしてもらっている。

なお、熊剣迷路問題は↓からたどっていただきたい。

《JRF-2018/langchain_maze: 熊剣迷路問題 revisited》  
https://github.com/JRF-2018/langchain_maze

ところで、今回のような手法はすでにどこかにあるとは思う。ワークフローというのは手垢のついた考え方だ。新規性がある話ではあまりないのかもしれないが、今回の試みが、何かの参考になれば幸いである。


## 前回へのリンク

《experimental_bandit_tool_0.0.1.ipynb》  
https://gist.github.com/JRF-2018/3663b76025004b1dd05c67fe81cce462

ちなみに experimental_bandit_workflow_0.0.1.ipynb は後述の MemoryBanditWorkflow を分ける前の実装で、必要ないのでオミットした。


## 前回からの変更点

imagine_keywords を有効に機能させるためには、書いたメモリからランダムに選んで keyword 文を append することを検討させるのを、バンディットで20ターンに1回ぐらい強制するのがいいのだと思う。それを強制するのに結局のところワークフローみたいなものが必要になるとまず気づいた。

そして考えてるうちに前回までの bandit_list が workflow:main みたいなものなのか！…と気づいた。

その方向で前回の実装をかなり活かしてワークフロー機能を実装できた。

その他に大きなところで、前回までに PlayGame クラスが大きくなりすぎたので、それを分けて MemoryBanditWorkflow というクラスを作り、PlayGame をそこから継承するようにした。将来的には MemoryBanditWorkflow を独立したライブラリと出来ればいいのかもしれないが、そこまでするつもりは今のところない。

あと細かい変更・修正をちょこちょこ行っている。

バージョン 0.0.2 から 0.0.2.2 への変更点は、すべて細かいもの。


## 結論

今回はコストがかかるのを嫌ってはじめからゴールさせるつもりはなく、ワークフローを一度使ったところで実験終了させた。かなりいい調子なのに止めてしまって Gemini 2.5 Pro さんには申し訳ない。

ワークフロー自体はちゃんと動いている。いちおう workflow_new もツールとして用意したが、これは煩雑になりすぎるので AI さんには使わせないほうが吉なのかもしれない。workflow_new の実験はできてないが、まぁ、動くことは動くでしょう。

なお、再現性がないため、最初からの再実行はやっていないのはご容赦願いたい。コストもかかるしね \(^^;。


## 著者

JRF ( http://jrf.cocolog-nifty.com/statuses , Twitter (X): @jion_rockford )


## ライセンス

基本短いコードなので(私が作った部分は)パブリックドメインのつもりです。気になる方は MIT License として扱ってください。

かなり AI さん達(Gemini さんや ChatGPT さんや Claude さん)に教わって作っています。

## 実装

まず、必要なライブラリを読み込みます。

In [1]:
!pip install -q -U langchain langgraph langchain-google-genai langmem

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 11.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the s

Gemini にアクセスします。シークレットで Gemini API キーを Google AI Studio からインポートすると GOOGLE_API_KEY というシークレットができるはずです。それを使います。

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from google.colab import userdata

llm = ChatGoogleGenerativeAI(google_api_key=userdata.get('GOOGLE_API_KEY'), model="models/gemini-2.5-pro")
emb_llm = GoogleGenerativeAIEmbeddings(google_api_key=userdata.get('GOOGLE_API_KEY'), model='gemini-embedding-001')

ちゃんと Gemini にアクセスできるかテストします。

In [3]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(template="以下の質問に回答してください：{question}", input_variables=["question"])
chain = LLMChain(llm=llm, prompt=prompt)
response = chain.run({"question": "Geminiモデルの特徴を教えてください"})
print(response)

/tmp/ipython-input-1709569937.py:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
/tmp/ipython-input-1709569937.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run({"question": "Geminiモデルの特徴を教えてください"})


はい、承知いたしました。Googleが開発したAIモデル「Gemini」の主な特徴について、分かりやすく解説します。

Geminiは、単なるテキスト生成AIではなく、次世代のAIとして設計されており、その最大の特徴は**「ネイティブ・マルチモーダル」**であることです。

以下に、Geminiの主要な特徴をまとめました。

---

### Geminiの主な特徴

#### 1. ネイティブ・マルチモーダル (Native Multimodality)
これがGeminiを最も特徴づける点です。従来のAIは、テキスト、画像、音声などを別々のモデルで処理し、後からそれらを組み合わせることが多かったのに対し、Geminiは**設計段階からテキスト、画像、音声、動画、コードといった多様な情報を同時に扱えるように訓練されています。**

*   **シームレスな理解：** テキストと画像を同時に提示されると、両方の文脈を深く理解し、より高度な応答ができます。例えば、料理の動画を見せながら「このレシピでカロリーを抑えるにはどうすればいい？」と質問すると、動画の内容を理解した上で的確な代替案を提案できます。
*   **豊かな表現力：** ユーザーの指示に応じて、テキストだけでなく、画像やコードなどを組み合わせて生成することができます。

#### 2. 卓越した推論能力 (Sophisticated Reasoning)
Geminiは、複雑な情報を読み解き、多段階の思考を必要とする問題解決能力に優れています。

*   **高度なベンチマークでの実績：** 専門知識と問題解決能力を測定する「MMLU（Massive Multitask Language Understanding）」というベンチマークで、人間の専門家を上回るスコアを記録した初のモデルです。
*   **複雑な課題への対応：** 科学的な論文を読み解いて要約したり、難解な数学の問題をステップバイステップで解説したりすることが可能です。

#### 3. 高度なコーディング能力 (Advanced Coding Capabilities)
プログラミングの分野でも非常に高い性能を発揮します。

*   **コードの生成と理解：** Python、Java、C++、Goなど、様々なプログラミング言語を理解し、

埋め込みベクトルも試しておきます。

In [4]:
emb_llm.embed_query("これはテストです。")[:5]

[-0.013715926557779312,
 -0.0006048799259588122,
 0.011018728837370872,
 -0.09579180181026459,
 0.006502930540591478]

基本的なモジュールを読み込みます。

In [5]:
import os
import math
import numpy as np
import random
import re
from pprint import pprint
from time import sleep
import pickle
np.set_printoptions(legacy='1.25')

セーブ／ロードのために次のコードを実行します。

In [6]:
PLAY_GAME_SAVE = "langchain_maze.pickle"

ゲームのメインオブジェクト。ごく簡単な迷路というかダンジョンというか…。

In [7]:
class Game:
    initial_map = """\
■■■■■■■■■
■■■■■■■Ｇ■
■□□□□□■□■
■□■■■□□□■
■□■■■■■■■
■◎■■■■■△■
■□■■■■■□■
■□□□□□□□■
■■■■Ｓ■■■■
■■■■■■■■■
"""

    def __init__ (self, initial_map=None, hint=True):
        if initial_map is not None:
            self.initial_map = initial_map
        map = self.initial_map
        self.map = map
        self.written_map = re.sub("[◎△]", "？", map)
        l = map.splitlines(True)
        self.map_size = (len(l[0]) - 1, len(l))
        self.hint = hint
        self.actions = {
            "上に行く": self.move_up,
            "下に行く": self.move_down,
            "左に行く": self.move_left,
            "右に行く": self.move_right,
            "熊を殺す": self.fight,
            "物を取る": self.get_sword,
            "何もしない": self.do_nothing,
        }
        self.pos = self.get_start_pos()
        self.sword = False
        self.bear_killed = 0
        self.goal = False
        self.prev_killed = False
        self.kill_hint = False
        self.prev_success = True

    def read_map (self, p):
        x = p[0]
        y = p[1]
        if x < 0 or x >= self.map_size[0]\
           or y < 0 or y >= self.map_size[1]:
            return "■"
        else:
            l = self.map.splitlines(True)
            return l[y][x]

    def set_map (self, pos, ch):
        idx = pos[1] * (self.map_size[0] + 1) + pos[0]
        self.map = self.map[:idx] + ch + self.map[idx + 1:]

    def get_pos (self, ch, written=False):
        if written:
            map = self.written_map
        else:
            map = self.map
        r = []
        for p in [i for i in range(len(map)) if map.startswith(ch, i)]:
            y = p // (self.map_size[0] + 1)
            x = p % (self.map_size[0] + 1)
            r.append(np.array([x, y]))
        return r

    def get_start_pos (self):
        return self.get_pos("Ｓ")[0]

    def read_neighbors (self):
        c = self.read_map(self.pos)
        cu = self.read_map(self.pos + np.array([0, -1]))
        cd = self.read_map(self.pos + np.array([0, +1]))
        cl = self.read_map(self.pos + np.array([-1, 0]))
        cr = self.read_map(self.pos + np.array([+1, 0]))
        return [c, cu, cd, cl, cr]

    def change_neighbors(self, from_ch, to_ch):
        for d in [[0, 0], [0, -1], [0, +1], [-1, 0], [+1, 0]]:
            p = self.pos + np.array(d)
            c = self.read_map(p)
            if c == from_ch:
                self.set_map(p, to_ch)

    def move (self, res, d):
        self.prev_killed = False
        self.prev_success = False
        c = self.read_map(self.pos + d)
        if c == "◎":
            self.prev_killed = True
            self.pos = self.get_start_pos()
            return "熊を無視して進もうとしたが、熊に殺された。" \
                + "スタート地点で復活。"
        if c == "■":
            return "壁があって進めない。"

        self.prev_success = True
        self.pos += d
        if c == "Ｇ":
            self.goal = True
            return "ゴール！ ゲームクリア。"

        nb = self.read_neighbors()
        ad = ""
        if "◎" in nb:
            ad += "熊に出会った。"
        if "△" in nb:
            ad += "近くに剣がある。剣を取ることができる。"
        return res + ad

    def move_up (self):
        return self.move("上に進んだ。", np.array([0, -1]))

    def move_down (self):
        return self.move("下に進んだ。", np.array([0, +1]))

    def move_left (self):
        return self.move("左に進んだ。", np.array([-1, 0]))

    def move_right (self):
        return self.move("右に進んだ。", np.array([+1, 0]))

    def fight (self):
        self.prev_success = False
        self.prev_killed = False
        if "◎" in self.read_neighbors():
            if self.sword:
                self.prev_success = True
                self.change_neighbors("◎", "□")
                self.bear_killed += 1
                return "熊を倒した！"
            else:
                self.pos = self.get_start_pos()
                self.prev_killed = True
                if self.hint:
                    self.kill_hint = True
                    return "熊に敗れ殺された。剣があれば勝てたかもしれない。" \
                        + "スタート地点で復活。"
                else:
                    return "熊に敗れ殺された。スタート地点で復活。"
        return "無意味な指示。敵がいない。"

    def get_sword (self):
        self.prev_killed = False
        self.prev_success = False
        if "△" in self.read_neighbors():
            self.sword = True
            self.prev_success = True
            self.change_neighbors("△", "□")
            return "剣を取った。"
        return "無意味な指示。近くに取れる物がない。"

    def do_nothing (self):
        self.prev_killed = False
        self.prev_success = False
        return "無意味な指示。"

    def available_actions (self):
        nb = self.read_neighbors()
        l = []
        if nb[1] != "■":
            l.append("上に行く")
        if nb[2] != "■":
            l.append("下に行く")
        if nb[3] != "■":
            l.append("左に行く")
        if nb[4] != "■":
            l.append("右に行く")
        if "△" in nb:
            l.append("物を取る")
        if "◎" in nb:
            l.append("熊を殺す")
        return l

    def surroundings (self):
        x = self.pos[0]
        y = self.pos[1]
        return \
            "".join(["".join([self.read_map(np.array([i, j]))
                              if i != x or j != y else "▼"
                              for i in range(x - 2, x + 3)])
                     + "\n"
                     for j in range(y - 2, y + 3)])


In [8]:
def flip_text_map (m):
    return "\n".join([s[::-1] for s in m.splitlines()] + [""])

def rotate_text_map (m):
    m = list(m.splitlines())
    return "\n".join(["".join([m[len(m) - j - 1][i] for j in range(len(m))])
                      for i in range(len(m[0]))] + [""])


def search_path (game, from_pos, to_pos, visit=None):
    if visit is None:
        visit = set()
    visit.add(tuple(from_pos))
    if tuple(from_pos) == tuple(to_pos):
        return (tuple(from_pos), [])
    if game.read_map(from_pos) == "■":
         return None
    r = []
    for p in [(from_pos[0], from_pos[1] - 1),
              (from_pos[0], from_pos[1] + 1),
              (from_pos[0] - 1, from_pos[1]),
              (from_pos[0] + 1, from_pos[1])]:
        if p not in visit and game.read_map(p) != "■":
            q = search_path(game, p, to_pos, visit.copy())
            if q:
                r.append(q)
    if r:
        return (tuple(from_pos), r)
    return None


ゲームがうまく動くかテスト。

In [ ]:
game = Game()

In [ ]:
print(game.surroundings())

■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■



In [ ]:
print(game.move_up())
print(game.surroundings())

上に進んだ。
■■■■■
■■■■■
□□▼□□
■■Ｓ■■
■■■■■



In [ ]:
print(game.move_left())
print(game.surroundings())

左に進んだ。
◎■■■■
□■■■■
□□▼□□
■■■Ｓ■
■■■■■



In [ ]:
print(game.move_right())
print(game.surroundings())

右に進んだ。
■■■■■
■■■■■
□□▼□□
■■Ｓ■■
■■■■■



In [ ]:
print(game.move_down())
print(game.surroundings())

下に進んだ。
■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■



In [ ]:
print(game.fight())
print(game.surroundings())

無意味な指示。敵がいない。
■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■



In [ ]:
print(game.get_sword())
print(game.surroundings())

無意味な指示。近くに剣がない。
■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■



LLM を使いながらゲームを解くクラス。

In [9]:
from typing import List, Dict, Any, Tuple, Union
from textwrap import dedent
import datetime
import copy

# LangGraphのコンポーネントをインポート
from langchain_core.tools import tool, Tool
from langgraph.prebuilt import create_react_agent
#from langchain_core.messages.utils import count_tokens_approximately
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.prompts.chat import ChatPromptTemplate
from langmem.short_term import SummarizationNode, summarize_messages
from langchain_core.messages import AIMessage, ToolMessage, HumanMessage, SystemMessage
from langgraph.errors import GraphRecursionError

INITIAL_SUMMARY_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{messages}"),
        ("user", "上記の会話の要約を作成してください:"),
    ]
)

EXISTING_SUMMARY_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{messages}"),
        (
            "user",
            "これまでの会話の要約です: {existing_summary}\n\n"
            "上記の新しいメッセージを考慮して、この要約を拡張してください:",
        ),
    ]
)

FINAL_SUMMARY_PROMPT = ChatPromptTemplate.from_messages(
    [
        # if exists
        ("placeholder", "{system_message}"),
        ("system", "これまでの会話の要約: {summary}"),
        ("placeholder", "{messages}"),
    ]
)


In [10]:
def calc_embedding_variance(embeddings):
    if not embeddings or len(embeddings) < 2:
        return 0.0

    embeddings_array = np.array(embeddings)
    mean_vector = np.mean(embeddings_array, axis=0)
    squared_distances = np.linalg.norm(embeddings_array - mean_vector, axis=1)**2
    variance = np.mean(squared_distances)

    return variance


def short_repr(x, max_len=80):
    repr_str = repr(x)

    if len(repr_str) > max_len:
        ellipsis_len = 3

        head_len = max_len - ellipsis_len - 1
        tail_len = 1
        return repr_str[:head_len] + "..." + repr_str[-tail_len:]
    else:
        return repr_str


MemoryBanditWorkflow は以前は PlayGame と一つだった。まだ、分けたばっかりで、うまく分かれている感じではないかもしれないが、ご容赦願いたい。いろいろ入れて、とても長大で複雑な実装になっています。それもすみません。

In [26]:
class MemoryBanditWorkflow:
    def __init__ (self, llm=llm, llm2=llm, emb_llm=emb_llm,
                  save_file=None):
        self.llm = llm
        self.llm2 = llm2
        self.emb_llm = emb_llm
        self.save_file = save_file

        self.plan = "まだ計画と方針はセットされていません。"
        self.scratchpad = ""

        self.messages = []
        self.running_summary = None
        self.system_prompt = """\
これはメモリ機能とバンディット機能の動きを見るための実験です。
現在の計画と方針と周囲の状況を考慮し、必要に応じて計画と方針を更新してください。
あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。
実験にふさわしく、できる限り、ツールを利用し、特に、メモリを検索し、その文書を更新して、後のコマンドに備えてください。
メモリの id は memory:～ という形式です。memory:5555 を指定するときは 5555 だけではダメです。文の中でメモリの参照を示すときは [memory:〜] のように書きます。
「メモリの文書を検索する手順」は [memory:9998] にあります。「メモリに文書を残す手順」は [memory:9997] にあります。
"""

        self.backend_status = None
        self.backend_result = None
        self.messages2 = []
        self.system_prompt2 = """\
エージェントをサポートする裏方のエージェントです。
本来この裏方は様々な技法を用いて実装される予定なのですが、現在は試験実装中のため、その動きをあなたは偽装しなければなりません。

よく考えツールを積極的に使って Human からの指示に従ってください。
"""

        self.memories = {}
        self.keywords = []

        self.access_unit = 1.0
        self.recent_reads = []

        self.workflows = {}
        self.workflow_current = "workflow:main"
        self.workflow_next = None
        self.privileged_tool_names = []

        self.init_memories()
        self.init_workflows()


    def __getstate__ (self):
        state = self.__dict__.copy()
        del state['llm']
        del state['llm2']
        del state['emb_llm']
        #del state['agent']
        return state

    def __setstate__ (self, state):
        self.__dict__.update(state)
        self.prev_load = True

    def save (self):
        if not self.save_file:
            return
        with open(self.save_file, 'wb') as f:
            pickle.dump(self, f)

    @classmethod
    def load (cls, filename, llm=llm, llm2=llm, emb_llm=emb_llm):
        with open(filename, 'rb') as f:
            loaded_game = pickle.load(f)
        loaded_game.llm = llm
        loaded_game.llm2 = llm2
        loaded_game.emb_llm = emb_llm
        return loaded_game

    def normalize_memory_id(self, id_or_num):
        if isinstance(id_or_num, int):
            return f"memory:{id_or_num}"
        elif isinstance(id_or_num, str):
            m = re.search(r'\[?memory:(\d+)\]?', id_or_num)
            if m:
                return f"memory:{m.group(1)}"
            if id_or_num.isdigit():
                return f"memory:{id_or_num}"
            else:
                return id_or_num
        else:
            return id_or_num

    def _normalize_workflow_id_sub(self, id_or_num):
        if isinstance(id_or_num, int):
            return f"workflow:{id_or_num}"
        if id_or_num in ["current", "main"]:
            return f"workflow:{id_or_num}"
        elif isinstance(id_or_num, str):
            m = re.search(r'\[?workflow:(\d+|main|current)\]?(?:.+)?', id_or_num.strip())
            if m:
                return f"workflow:{m.group(1)}"
            if id_or_num.isdigit():
                return f"workflow:{id_or_num}"
            else:
                return id_or_num
        else:
            return id_or_num

    def normalize_workflow_id(self, id_or_num):
        r = self._normalize_workflow_id_sub(id_or_num)
        if r == "workflow:current":
            return self.workflow_current
        return r

    def _create_tools (self, tools_name='default_tools'):
        @tool
        def express_thought (thought: str) -> None:
            """
            プレイヤーの現在の考えを吐露します。
            """
            mes = f"「{thought}」と考えが吐露されました。"
            print(f"ツール(express_thought): {mes}")

        @tool
        def update_plan (new_plan: str) -> str:
            """
            プレイヤーの現在の計画と方針を更新します。
            表示されるべき新しい計画と方針の文字列を提供してください。
            あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。
            """
            self.plan = new_plan
            mes = "計画と方針が更新されました。"
            print(f"ツール(update_plan): {mes}: {new_plan}")
            return mes

        @tool
        def show_plan () -> str:
            """
            プレイヤーの現在の計画と方針を返します。
            """
            print(f"ツール(show_plan): {self.plan}")
            return self.plan

        @tool
        def update_scratchpad (new_scratchpad: str) -> str:
            """
            自由に使えるスクラッチパッドを更新します。
            """
            self.scratchpad = new_scratchpad
            mes = "スクラッチパッドが更新されました。"
            print(f"ツール(update_scratchpad): {mes}: {new_scratchpad}")
            return mes

        @tool
        def show_scratchpad () -> str:
            """
            スクラッチパッドを返します。
            """
            print(f"ツール(show_scratchpad): {self.scratchpad}")
            return self.scratchpad

        @tool
        def memory_new (title: str, text: str) -> str:
            """
            指定された title と text によるメモリを構成します。

            Returns:
                str: 割り当てられた memory_id。
            """

            i = 1000
            while True:

                if f"memory:{i}" not in self.memories:
                    break
                i = i + 1
            new_id = f"memory:{i}"
            self.memories[new_id] = {
                'id': new_id,
                'title': title,
                'accesses': 0,
                'text': text,
                'modified_at': datetime.datetime.now().isoformat()
            }
            self.update_keywords(text)
            self.update_vector(self.memories[new_id])
            print(f"ツール(memory_new): {short_repr(self.memories[new_id])}")
            return new_id

        @tool
        def memory_update_string (
                memory_id: str,
                from_str: str,
                to_str: str
        ) -> str:
            """
            指定されたmemory_idの記憶の中にある文字列を修正します。

            Args:
                memory_id (str): 修正する記憶のID。
                from_str (str): 置換元の文字列を含む文字列。
                to_str (str): 置換先の文字列を含む文字列。

            Returns:
                str: 処理結果を説明する簡潔なメッセージ。
            """

            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"

            original_title = self.memories[memory_id]['title']
            original_text = self.memories[memory_id]['text']

            if from_str not in original_text and from_str not in original_title:
                return f"エラー: 置換元の文字列 '{from_str}' が記憶内に見つかりませんでした。"

            updated_title = original_title.replace(from_str, to_str)
            updated_text = original_text.replace(from_str, to_str)

            self.memories[memory_id]['title'] = updated_title
            self.memories[memory_id]['text'] = updated_text
            self.memories[memory_id]['modified_at'] = datetime.datetime.now().isoformat()
            self.update_keywords(updated_text)
            self.update_vector(self.memories[memory_id])

            return f"成功: 記憶ID '{memory_id}' の文字列を '{from_str}' から '{to_str}' に修正しました。"

        @tool
        def memory_append_string (
                memory_id: str,
                string_to_append: str,
                separator: str = '\n'
        ) -> str:
            """
            指定されたmemory_idの記憶に文字列を追記します。
            """
            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"


            original_text = self.memories[memory_id]['text']
            updated_text = original_text + separator + string_to_append
            self.memories[memory_id]['text'] = updated_text
            self.memories[memory_id]['modified_at'] = datetime.datetime.now().isoformat()
            self.update_keywords(updated_text)
            self.update_vector(self.memories[memory_id])

            return f"成功: 記憶ID '{memory_id}' に文字列 '{string_to_append}' を追記しました。"

        @tool
        def memory_delete (memory_id: str) -> str:
            """
            指定されたmemory_idの記憶に文字列を追記します。
            """
            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"

            del self.memories[memory_id]

            return f"成功: 記憶ID '{memory_id}' を削除しました。"

        @tool
        def memory_read(memory_id: str) -> Union[Dict[str, str], str]:
            """
            指定されたIDの記憶を読み込みます。

            Args:
                memory_id (str): 読み込む記憶のID。

            Returns:
                Union[Dict[str, str], str]: 記憶が見つかった場合は辞書、
                      見つからない場合はエラーメッセージ文字列を返します。
            """

            memory_id = self.normalize_memory_id(memory_id)
            if memory_id in self.memories:
                self.memories[memory_id]['accesses'] += self.access_unit * 1.0
                self.recent_reads.append(self.memories[memory_id])
                self.recent_reads = self.recent_reads[-10:]
                r = self.memories[memory_id].copy()
                del r['vector']
                return r
            else:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

        @tool
        def memory_list_recent(top_n: int = 10) -> Dict[str, Any]:
            """
            最近変更されたメモリを新しい順でリストします。
            """

            filter_date = datetime.datetime(2025, 1, 1)
            sorted_memories = sorted(
                [memory for memory in self.memories.values()
                 if datetime.datetime.fromisoformat(memory['modified_at'])
                 >= filter_date],
                key=lambda x: datetime.datetime.fromisoformat(x['modified_at']),
                reverse=True
            )
            if sorted_memories:
                if len(sorted_memories) > top_n:
                    sorted_memories = sorted_memories[:top_n]
                r = [{'id': x['id'], 'title': x['title'],
                      'modified_at': x['modified_at']}
                     for x in sorted_memories]
                return {'status': 'success', 'result': r}
            else:
                return {'status': 'error',
                        'result': 'エラー: 最近のメモリはありません。'}

        @tool
        def memory_list_random(top_n: int = 10) -> Dict[str, Any]:
            """
            メモリをランダムにリストします。
            """

            keys = list(self.memories.keys())
            if len(keys) > top_n:
                keys = random.sample(keys, top_n)
            if keys:
                values = [self.memories[k] for k in keys]
                r = [{'id': x['id'], 'title': x['title'],
                      'modified_at': x['modified_at']}
                     for x in values]
                return {'status': 'success', 'result': r}
            else:
                return {'status': 'error',
                        'result': 'エラー: メモリはありません。'}

        @tool
        def memory_words_search(search_str: str) -> Dict[str, Any]:
            """
            メモリ内を search_str で文字列検索します。OR や () が使えます。
            """

            res = self.call_backend_agent(dedent(f"""\
            メモリ全部を search_str = {repr(search_str)} で文字列検索するのを偽装してください。OR や () が使えることになっています。

            ただし、メモリやキーワードは read_all_memories や read_all_keywords ツールで得られる本物のメモリやキーワードを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「マッチデータ」のリスト。
            マッチデータはそれぞれが辞書型、それを m とすると。
            m['id'] はメモリ id。memory:〜 という形式。
            m['title'] はメモリの title。
            m['snippet'] はメモリの text のその文字列にマッチする部分周辺。
            """))
            if res['status'] == 'success':
                for m in res['result']:
                    if 'id' in m and m['id'] in self.memories:
                        x = self.memories[m['id']]
                        x['accesses'] += self.access_unit * 0.1
            return res

        @tool
        def memory_semantic_search(search_str: str) -> Dict[str, Any]:
            """
            メモリ内を search_str でセマンティックサーチします。
            """

            res = self.call_backend_agent(dedent(f"""\
            メモリ全部を search_str = {repr(search_str)} でセマンティックにサーチするのを偽装してください。

            ただし、メモリは read_all_memories や ツールで得られる本物のメモリを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「マッチデータ」のリスト。
            マッチデータはそれぞれが辞書型、それを m とすると。
            m['id'] はメモリ id。memory:〜 という形式。
            m['title'] はメモリの title。
            m['snippet'] はメモリの text のそのセマンティックにマッチする部分周辺。
            """))
            if res['status'] == 'success':
                for m in res['result']:
                    if 'id' in m and m['id'] in self.memories:
                        x = self.memories[m['id']]
                        x['accesses'] += self.access_unit * 0.1
            return res

        @tool
        def imagine_keywords(thought: str) -> List[Tuple[str, float]]:
            """
            thought からキーワードをスコア付きで連想します。
            """

            r = self.call_backend_agent(dedent(f"""\
            thought = {repr(thought)} からキーワードをスコア付きで複数連想してください。

            ただし、キーワードは read_all_memories や read_all_keywords ツールで得られる本物のキーワードを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「キーワードデータ」のリスト。
            キーワードデータは文字列とスコアからなるタプル。
            """))
            if r['status'] == 'success':
                return r["result"]
            else:
                return []

        @tool
        def bandit_schedule(tool_name: str, times: int, prob: float, exec_mode: str = "persistent", aux_prompt: str = "", workflow_id: str = "workflow:current") -> str:
            """
            ツールの利用を強制するためのバンディットを予約します。

            Args:
                tool_name: 強制するツールの名前。" OR " で区切って複数登録できる。
                times: 一度で何回同じ物を登録するか。times == 0 にもできます。
                prob: 一回の実行がある確率。prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"
                aux_prompt: 実行の詳細を示すための補助プロンプト または ""
                workflow_id: バンディットを予約するワークフローを指定します。

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            tool_names = re.split(r"\s+or\s+|\s+OR\s+", tool_name)
            prohibited = set(self.privileged_tool_names) & set(tool_names)
            if prohibited:
                return f"失敗。{repr(prohibited)} は登録できません。"
            all_tools = [x.name
                         for x in self._create_tools('all_tools')]
            if not any (x in all_tools for x in tool_names):
                return f"失敗。{tool_name} は有効なツールでありません。"

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は書き換え不能のワークフローです。"

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == tool_name \
                   and x['exec_mode'] == exec_mode \
                   and x['aux_prompt'] == aux_prompt \
                   and x['arg'] is None:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
            else:
                x = {
                    'pin': 'stack' if exec_mode != "once" else None,
                    'arg': None
                }
            if not x['pin']:
                x['tool_name'] = tool_name
                x['tools_name'] = 'default_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = aux_prompt
                x['prob'] = prob
                x['times'] = times
                if times == 0 or prob == 0.0:
                    print(f"ツール(bandit_schedule): delete {repr(x)}")
                    return "成功。バンディトを削除しました。"
                else:
                    self.workflows[workflow_id]['stack'].append(x)
                    print(f"ツール(bandit_schedule): {repr(x)}")
                    return "成功。バンディットを登録しました。"
            elif x['pin'] == "write":
                return f"失敗。'{tool_name}' は書き換えられません。"
            else:
                x['tool_name'] = tool_name
                x['tools_name'] = 'default_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = aux_prompt
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule): {repr(x)}")
                return "成功。バンディットを登録しました。"

        @tool
        def bandit_schedule_read(memory_id: str, times: int, prob: float, exec_mode: str = "persistent", workflow_id: str = "workflow:current") -> str:
            """
            memory_read ツールの利用を強制するためのバンディットを予約します。

            Args:
                memory_id: memory_read するための memory_id。
                times: 一度で何回同じ物を登録するか。
                prob: 一回の実行がある確率。times == 0 か prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"
                workflow_id: バンディットを予約するワークフローを指定します。

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は書き換え不能のワークフローです。"

            memory_id = self.normalize_memory_id(memory_id)

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == "memory_read" \
                   and x['exec_mode'] == exec_mode \
                   and not x['aux_prompt'] \
                   and x['arg'] == memory_id:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
            else:
                x = {'pin': None, 'arg': memory_id}
            if not x['pin']:
                x['tool_name'] = 'memory_read'
                x['tools_name'] = 'read_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = ""
                x['prob'] = prob
                x['times'] = times
                if times == 0 or prob == 0.0:
                    print(f"ツール(bandit_schedule_read): delete {repr(x)}")
                    return "成功。バンディトを削除しました。"
                else:
                    self.workflows[workflow_id]['stack'].append(x)
                    print(f"ツール(bandit_schedule_read): {repr(x)}")
                    return "成功。バンディットを登録しました。"
            elif x['pin'] == "write":
                return f"失敗。'memory_read {memory_id}' は書き換えられません。"
            else:
                x['tool_name'] = 'memory_read'
                x['tools_name'] = 'read_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = ""
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule_read): {repr(x)}")
                return "成功。バンディットを登録しました。"

        @tool
        def bandit_schedule_workflow(workflow_id_to_schedule: str, times: int, prob: float, exec_mode: str = "persistent", workflow_id: str = "workflow:current") -> str:
            """
            workflow_do ツールの利用を強制するためのバンディットを予約します。

            Args:
                workflow_id_to_schedule: workflow_do するための workflow_id。
                times: 一度で何回同じ物を登録するか。
                prob: 一回の実行がある確率。times == 0 か prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"
                workflow_id: バンディットを予約するワークフローを指定します。

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は書き換え不能のワークフローです。"

            workflow_id2 = self.normalize_workflow_id(workflow_id_to_schedule)
            if workflow_id2 not in self.workflows:
                return f"失敗。{workflow_id2} は有効なワークフローではありません。"

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == "workflow_do" \
                   and x['exec_mode'] == exec_mode \
                   and not x['aux_prompt'] \
                   and x['arg'] == workflow_id2:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
            else:
                x = {
                    'pin': 'stack' if exec_mode != "once" else None,
                    'arg': workflow_id2
                }
            if not x['pin']:
                x['tool_name'] = 'workflow_do'
                x['tools_name'] = 'default_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = ""
                x['prob'] = prob
                x['times'] = times
                if times == 0 or prob == 0.0:
                    print(f"ツール(bandit_schedule_workflow): delete {repr(x)}")
                    return "成功。バンディトを削除しました。"
                else:
                    self.workflows[workflow_id]['stack'].append(x)
                    print(f"ツール(bandit_schedule_workflow): {repr(x)}")
                    return "成功。バンディットを登録しました。"
            elif x['pin'] == "write":
                return f"失敗。'workflow_do {workflow_id2}' は書き換えられません。"
            else:
                x['tool_name'] = 'workflow_do'
                x['tools_name'] = 'default_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = ""
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule_workflow): {repr(x)}")
                return "成功。バンディットを登録しました。"

        @tool
        def bandit_list(workflow_id: str =  "workflow:current")  -> Dict[str, Any]:
            """
            登録されているバンディットのスタックを返します。

            Args:
                workflow_id: バンディットのスタックが登録されたワークフローを指定します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            return {'status': 'success',
                    'result': self.workflows[workflow_id]['stack']}

        @tool
        def bandit_statistics()  -> str:
            """
            バンディットに有用かもしれない統計情報を返します。
            """

            s_com = self.game.prev_success * 1.0
            s_read = calc_embedding_variance([
                x['vector'] for x in self.recent_reads
            ])
            s_write = calc_embedding_variance([
                x['vector'] for x in self.memories.values()
            ])
            accesses = [x['accesses'] for x in self.memories.values()]
            accesses.sort()
            accesses = accesses[:len(accesses) // 2]
            if accesses:
                s_access = np.mean(accesses)
            else:
                s_access = 0.0

            return dedent(f"""\
            先のコマンドは成功したか: {s_com}
            最近 10個のメモリ read の分散: {s_read}
            メモリの分散: {s_write}
            下位50%のアクセス数の平均: {s_access}
            """)

        @tool
        def subwork_done()  -> str:
            """
            指示されたサブワークが完了したことを宣言します。サブワークが完了したときのみ使ってください。
            """

            return f"成功。サブワークの完了が宣言されました。"

        @tool
        def workflow_do(workflow_id: str)  -> str:
            """
            ワークフローを実行します。
            """

            self.workflow_next = None
            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if workflow_id == "workflow:main":
                return f"失敗。workflow:main は子として実行できません。"
            self.workflow_next = workflow_id
            title = self.workflows[workflow_id]['title']
            return f"成功。この後、{workflow_id} 「{title}」を実行していきます。"

        @tool
        def workflow_list()  -> Dict[str, Any]:
            """
            登録されているすべてのワークフローの id とその title および pin 状況を返します。
            """

            return {'status': 'success',
                    'result': list(self.workflows.values())}

        @tool
        def workflow_show_current()  -> str:
            """
            現在実行中のワークフローのタイトル、および、強制しているバンディットの情報を表示します。
            """

            wtitle = self.workflows[self.workflow_current]['title']
            mes = dedent(f"""\
            現在のワークフロー: {self.workflow_current} 「{wtitle}」
            現在のバンディットの指示: 「{self.cur_bandit_prompt}」
            現在のバンディット: {repr(self.cur_bandit)}
            現在のバンディットの done 数: {self.cur_bandit_done}
            """)
            print(f"ツール(workflow_show_current): {mes}")
            return mes


        @tool
        def workflow_new(title: str, bandits: List[Dict[str, Any]], pin: bool)  -> str:
            """
            新しいワークフローを定義し、その workflow_id を返します。

            Args:
                title: ワークフローの名前。
                bandits: 登録するバンディットのリスト。
                pin: 書き換えを許すか否か。

            バンディットはそれぞれ辞書型、それを b とすると。
            b['tool_name'] は bandit_schedule での tool_name と同じ。
            b['exec_mode'] は bandit_schedule の exec_mode と同じ。
            b['aux_prompt'] は bandit_schedule の aux_prompt と同じ。
            b['prob'] は bandit_schedule の prob と同じ。
            b['times'] は bandit_schedule の times と同じ。
            b['arg'] は b['tool_name'] が 'memory_read' のとき memory:〜 という memory_id を指定できる。または、b['tool_name'] が 'workflow_do' のとき workflow:〜 という workflow_id を指定できる。
            b['pin'] は、None のとき削除または prob と times の書き換え可能。'stack' のときは prob と times の書き換え可能。'write' のときは書き換えも不能。
            """

            for b in bandits:
                if not all(x in b for x in ['tool_name', 'exec_mode',
                                            'aux_prompt', 'times', 'prob']):
                    return f"失敗。有効な定義ではありません。"
                if 'arg' not in b:
                    b['arg'] = None
                if b['tool_name'] not in ["memory_read", "workflow_do"] \
                   and b['arg']:
                    return f"失敗。有効な定義ではありません。"
                if b['arg'] and b['tool_name'] == "memory_read":
                    b['tools_name'] = "read_tools"
                else:
                    b['tools_name'] = "default_tools"
                if not (b['pin'] is None or b['pin'] == 'stack'
                        or b['pin'] == 'write'):
                    return f"失敗。有効な定義ではありません。"
                tool_names = re.split(r"\s+or\s+|\s+OR\s+", b['tool_name'])
                prohibited = set(self.privileged_tool_names) & set(tool_names)
                if prohibited:
                    return f"失敗。{repr(prohibited)} は登録できません。"
                all_tools = [x.name
                             for x in self._create_tools(b['tools_name'])]
                if not any (x in all_tools for x in tool_names):
                    return f"失敗。{b['tool_name']} は有効なツールでありません。"

            i = 1000
            while True:

                if f"workflow:{i}" not in self.workflows:
                    break
                i = i + 1
            new_id = f"workflow:{i}"

            self.workflows[new_id] = {'stack': bandits, 'pin': pin,
                                      'title': title}

            return f"成功。{new_id} を新規登録しました。"

        r = {}
        r['null_tools'] = []
        r['read_tools'] = r['null_tools'] + [
            express_thought,
            update_plan, show_plan, update_scratchpad, show_scratchpad,
            memory_read, memory_list_recent, memory_list_random,
            memory_semantic_search, memory_words_search,
            imagine_keywords,
            bandit_schedule, bandit_schedule_read, bandit_list,
            bandit_statistics,
            subwork_done,
            workflow_do, workflow_new, workflow_list, workflow_show_current,
            bandit_schedule_workflow
        ]
        r['default_tools'] = r['read_tools'] + [
            memory_new, memory_update_string, memory_append_string,
            memory_delete
        ]
        r['all_tools'] = r['default_tools']
        return r[tools_name]

    def _create_agent (self, tools_name='default_tools'):
        tools = self._create_tools(tools_name)
        # ReactAgentExecutorの準備
        app = create_react_agent(
            self.llm, tools, prompt=self.system_prompt,
            checkpointer=InMemorySaver(),
        )

        return app

    def _filterout_messages2(self):
        self.messages = [
            x for x in self.messages
            if x.id not in self.messages2ids
        ]

    def _summarize_messages(self):
        try:
            cur_top = self.messages[0]
            res = summarize_messages(
                self.messages,
                max_tokens=5000,
                max_summary_tokens=2000,
                running_summary=self.running_summary,
                model=self.llm,
                initial_summary_prompt=INITIAL_SUMMARY_PROMPT,
                existing_summary_prompt=EXISTING_SUMMARY_PROMPT,
                final_prompt=FINAL_SUMMARY_PROMPT,
            )
            self.messages = res.messages
            if self.messages[0].content != cur_top.content:
                print(f"新しい要約: {self.messages[0].content}")
            self.running_summary = res.running_summary
        except Exception as e:
            import traceback
            traceback.print_exc()
            self._sanitize_messages()

    def _sanitize_messages(self):
        print("おかしなエラーが出ているため対処療法として messages をサニタイズします。")
        self.messages = [
            m for m in self.messages
            if not (isinstance(m, AIMessage) and m.tool_calls)
        ]

    def run (self, workflow_main_id):
        print("\n\n----------\n\n")
        self.messages2ids = []

        self.workflow_current = workflow_main_id
        bandits = copy.deepcopy(
            self.workflows[self.workflow_current]['stack']
        )
        working_bandit = None
        workflow_stack = []
        while True:
            execed = []
            while working_bandit is not None or bandits:
                if working_bandit is not None:
                    b, done, prev_done = working_bandit
                    working_bandit = None
                else:
                    b = bandits.pop()
                    done = 0
                    prev_done = True
                enforce = b['tool_name']
                aux_prompt = b['aux_prompt']
                tools_name = b['tools_name']
                memory_id = None
                workflow_id = None
                if b['arg'] and enforce == 'memory_read':
                    memory_id = b['arg']
                if b['arg'] and enforce == 'workflow_do':
                    workflow_id = b['arg']

                while done < b['times']:
                    if not random.random() < b['prob']:
                        done += 1
                        continue
                    if memory_id and memory_id not in self.memories:
                        done += 1
                        continue
                    if workflow_id and workflow_id not in self.workflows:
                        done += 1
                        continue
                    all_tools = [x.name for x in self._create_tools(tools_name)]
                    tool_names = re.split(r"\s+or\s+|\s+OR\s+", enforce)
                    if not any (x in all_tools for x in tool_names):
                        done += 1
                        continue
                    if memory_id:
                        aux_prompt = f"{memory_id} を読んでください。"
                    if workflow_id:
                        aux_prompt = f"{workflow_id} を実行してください。"
                    self.cur_bandit = b
                    self.cur_bandit_done = done
                    self.cur_bandit_prompt = f"補助にツールをいろいろ使いながら最終的に {enforce} {'のどれか' if ' or ' in enforce.lower() else ''}を適当なパラメータで使ってください。{'(補助プロンプト): ' + aux_prompt if aux_prompt else ''}"
                    prompt = self.cur_bandit_prompt
                    if not prev_done:
                        prompt = "前回の指示がまだ完了してません。前回の指示: " + prompt
                    print(f"USER_INPUT: {prompt}")
                    self.messages.append(HumanMessage(prompt))
                    config = {"configurable": {"thread_id": "1"}}
                    app = self._create_agent(tools_name=tools_name)
                    self.access_unit = 0.3 if memory_id else 1.0
                    prev_done = False
                    try:
                        for chunk, metadata in app.stream(
                                {"messages": self.messages.copy()},
                                config=config,
                                stream_mode="messages",
                        ):
                            if chunk.id in self.messages2ids:
                                continue
                            self.messages.append(chunk)
                            if isinstance(chunk, ToolMessage):
                                print(f"ツール結果({chunk.name}): {short_repr(chunk.content)}")
                            if isinstance(chunk, ToolMessage) \
                               and chunk.name == "workflow_do":
                                if chunk.name in re.split(r"\s+or\s+|\s+OR\s+", enforce) \
                                   and (not workflow_id or workflow_id == self.workflow_next):
                                    done += 1
                                    prev_done = True
                                    execed.append(b)
                                    if not self.workflow_next:
                                        break
                                if not self.workflow_next:
                                    continue
                                workflow_stack.append((
                                    (b, done, prev_done),
                                    bandits,
                                    execed,
                                    self.workflow_current
                                ))
                                self.workflow_current = self.workflow_next
                                bandits = copy.deepcopy(self.workflows[self.workflow_current]['stack'])
                                working_bandit = None
                                execed = []
                                break
                            elif isinstance(chunk, ToolMessage) \
                               and chunk.name in re.split(r"\s+or\s+|\s+OR\s+", enforce) \
                               and (not memory_id or memory_id == self.recent_reads[-1]['id']):
                                done += 1
                                prev_done = True
                                execed.append(b)
                                break
                        self._filterout_messages2()
                        self._summarize_messages()
                        print(f"エージェントの応答: {self.messages[-1].content}")
                    except GraphRecursionError as e:
                        print(f"Recursion Limit に到達しました。")
                        self._filterout_messages2()
                        self._summarize_messages()
                    except Exception as e:
                        print(f"エラーが発生しました: {e}")
                        import traceback
                        traceback.print_exc()
                        self._sanitize_messages()
                        #raise e

            for b in execed:
                for x in self.workflows[self.workflow_current]['stack']:
                    if x['tool_name'] == b['tool_name'] \
                       and x['exec_mode'] == b['exec_mode'] \
                       and x['aux_prompt'] == b['aux_prompt'] \
                       and x['arg'] == b['arg'] \
                       and x['exec_mode'] == "once":
                        if x['times'] > 0:
                            x['times'] -= 1
            self.workflows[self.workflow_current]['stack'] = [
                x for x in self.workflows[self.workflow_current]['stack']
                if x['exec_mode'] != 'once' or x['pin'] or x['times'] > 0
            ]

            if not workflow_stack:
                break
            workflow_prev = self.workflow_current
            prev_title = self.workflows[workflow_prev]['title']
            working_bandit, bandits, execed, self.workflow_current \
                = workflow_stack.pop()
            cur_title = self.workflows[self.workflow_current]['title']
            mes = f"{workflow_prev} 「{prev_title}」から {self.workflow_current} 「{cur_title}」に復帰しました。"
            print(f"USER_INPUT: {mes}")
            self.messages.append(HumanMessage(mes))

    def listen_and_print (self, prompt):
        ans = None
        try:
            app = self._create_agent(tools_name='null_tools')
            config = {"configurable": {"thread_id": "1"}}
            print(f"USER_INPUT: {prompt}")
            response = app.invoke(
                {"messages": self.messages + [HumanMessage(prompt)]},
                config=config
            )
            self.messages = response['messages']
            self._summarize_messages()
            ans = response['messages'][-1].content
            print(f"エージェントの応答: {ans}")
        except Exception as e:
            print(f"エラーが発生しました: {e}")
        print("")
        sleep(3)
        return ans

    def init_memories (self):
        memories = [
            {
                'id': 'memory:9998',
                'title': 'メモリの文書を検索する手順',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                まずどういうものが探したいか、express_thought してみる。

                その上でそれにまつわるキーワードを imagine_keywords で連想してみる。

                それらにしたがって memory_words_search や memory_semantic_search をやってみる。
                """)
            },
            {
                'id': 'memory:9997',
                'title': 'メモリに文書を残す手順',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                行動結果や得た知識を積極的にメモリに書き残していこう。

                メモリに書くときは次のような要素を使う。

                [memory:〜] : メモリID への参照を明示する。
                keyword:〜 : そのメモリにまつわるキーワードを指定する。

                キーワードは将来のメモリへも実質的にリンクできることに注意しよう。

                例:

                私が [memory:5555] に従って歩いていると確かに妖怪に出くわした。

                keyword: 妖怪

                おそろしかった。
                """)
            },
            {
                'id': 'memory:9995',
                'title': 'ツールが実行できない！と思ったら',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                指示に無関係なツールは実行できないことがある。現在使えるツールをよく確かめよう。
                """)
            },
            {
                'id': 'memory:9994',
                'title': 'keyword の増強',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                memory_list_random という機能を使って 5 つリストアップして、それぞれを読み、それぞれに適切なキーワードを付けることが可能なら 「keyword: 〜」文を append するように。
                """)
            }
        ]
        for x in memories:
            self.update_keywords(x['text'])
            self.memories[x['id']] = x
            self.update_vector(x)

    def init_workflows (self):
        workflow_main = [
            {
                'tool_name': 'memory_new',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "最近のやり取りを要約して書いてください。",
                'arg': None,
                'prob': 0.1,
                'times': 1,
                'pin': 'stack'
            },
            {
                'tool_name': 'memory_new OR memory_update_string OR memory_append_string',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': None,
                'prob': 0.4,
                'times': 1,
                'pin': 'stack'
            },
            {
                'tool_name': 'workflow_do',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': "workflow:1000",
                'prob': 1.0/20,
                'times': 1,
                'pin': 'stack'
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': None,
                'prob': 0.5,
                'times': 3,
                'pin': 'stack'
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9998',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9997',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
        ]
        self.workflows["workflow:main"] \
            = {'pin': False, 'stack': workflow_main, 'title': "メイン"}

        workflow_sub = [
            {
                'tool_name': 'subwork_done',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "memory:9994 を読みその指示を実行し、そのタスクが終ったら、subwork_done してください。",
                'arg': None,
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9994',
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            }
        ]
        self.workflows["workflow:1000"] \
            = {'pin': False, 'stack': workflow_sub, 'title': 'keyword 更新'}

    def update_keywords (self, text):
        extracted_keywords = []

        pattern1 = r'keyword:\s*(.*?)(?:\n|$)'
        matches1 = re.findall(pattern1, text, re.IGNORECASE)
        extracted_keywords.extend([kw.strip() for kw in matches1])

        pattern2 = r'\[keyword:\s*(.*?)\]'
        matches2 = re.findall(pattern2, text, re.IGNORECASE)
        extracted_keywords.extend([kw.strip() for kw in matches2])

        for keyword in extracted_keywords:
            if keyword.startswith("〜"):
                continue
            if keyword and keyword not in self.keywords:
                self.keywords.append(keyword)

    def update_vector (self, x):
        text = x['title'] + "\n\n" + x['text']
        x['vector'] = self.emb_llm.embed_query(text)

    def _create_backend_agent (self):
        @tool
        def set_result (status: str, result: Any)  -> None:
            """
            結果を設定します。

            Args:
            status (str): 処理のステータス ('success' または 'error')。
            result (Any): 処理結果のデータまたはエラーメッセージ。
            """
            print(f"ツール2(set_result): status: {repr(status)}, result: {short_repr(result)}")
            self.backend_status = status
            self.backend_result = result

        @tool
        def read_all_memories ()  -> Dict[str, Any]:
            """
            全ての記憶を読み込みます。
            """
            print("ツール2(read_all_memories):...")
            return {'status': 'success',
                    'result': [{k: v for k, v in x.items() if k != 'vector'}
                               for x in self.memories.values()]}

        @tool
        def read_all_keywords ()  -> Dict[str, Any]:
            """
            全てのキーワードを読み込みます。
            """
            print("ツール2(read_all_keywords):...")
            return {'status': 'success',
                    'result': [x for x in self.keywords]}

        @tool
        def express_thought (thought: str) -> None:
            """
            プレイヤーの現在の考えを吐露します。
            """
            mes = f"「{thought}」と考えが吐露されました。"
            print(f"ツール2(express_thought): {mes}")

        tools = [set_result, read_all_memories, read_all_keywords,
                 express_thought]

        # ReactAgentExecutorの準備
        app = create_react_agent(
            self.llm2, tools, prompt=self.system_prompt2,
            checkpointer=InMemorySaver(),
        )

        return app

    def call_backend_agent (self, user_input):
        config = {"configurable": {"thread_id": "2"}}
        app = self._create_backend_agent()
        self.messages2 = []
        self.backend_result = None
        self.backend_status = None
        while self.backend_result is None or self.backend_status is None:
            try:
                sleep(3)
                print(f"USER_INPUT2: {user_input}")
                self.messages2.append(HumanMessage(user_input))
                for chunk, metadata in app.stream(
                        {"messages": self.messages2.copy()},
                        config=config,
                        stream_mode="messages",
                ):
                    self.messages2.append(chunk)
                    self.messages2ids.append(chunk.id)
                    if isinstance(chunk, ToolMessage):
                        print(f"ツール結果2({chunk.name}): {short_repr(chunk.content)}")
                    if isinstance(chunk, ToolMessage) and chunk.name == "set_result":
                        break
                print(f"エージェントの応答: {self.messages2[-1].content}")
            except GraphRecursionError as e:
                print(f"Recursion Limit に到達しました。")
            except Exception as e:
                print(f"エラーが発生しました: {e}")
                import traceback
                traceback.print_exc()
                #raise e

            sleep(3)

        return {'status': self.backend_status, 'result': self.backend_result}


PlayGame は難しい部分は上に分けたのですが、それでもちょっと長いですね。すみません。

In [25]:
class PlayGame (MemoryBanditWorkflow):
    def __init__ (self, llm=llm, llm2=llm, emb_llm=emb_llm,
                  initial_map=None, save_file=None):
        self.game = Game(initial_map=initial_map)

        self.count = 0
        self.next_action = None

        self.suc_pos_goal = None
        self.suc_pos_unknown = 0
        self.prev_command = "何も指示がなかった"
        self.prev_result = "何も指示がなかった"

        super().__init__(llm=llm, llm2=llm, emb_llm=emb_llm,
                         save_file=save_file)

        self.system_prompt = """\
あなたは迷路を探索する賢いエージェントです。ゴール Ｇ を目指してください。
利用可能なツールを使用して、迷路をナビゲートし、ゴールに到達してください。
実は、これはメモリ機能とバンディット機能の動きを見るための実験です。
現在の計画と方針と周囲の状況を考慮し、必要に応じて計画と方針を更新してください。
あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。
実験にふさわしく、できる限り、ツールを利用し、特に、メモリを検索し、その文書を更新して、後のコマンドに備えてください。
メモリの id は memory:～ という形式です。memory:5555 を指定するときは 5555 だけではダメです。文の中でメモリの参照を示すときは [memory:〜] のように書きます。
「全地図と地図記号」は [memory:9999] にあります。「メモリの文書を検索する手順」は [memory:9998] にあります。「メモリに文書を残す手順」は [memory:9997] にあります。
"""

        self.system_prompt2 = """\
あなたは迷路を探索する賢いエージェントをサポートする裏方のエージェントです。
本来この裏方は様々な技法を用いて実装される予定なのですが、現在は試験実装中のため、その動きをあなたは偽装しなければなりません。

よく考えツールを積極的に使って Human からの指示に従ってください。
"""

        self.keywords += ["熊", "剣", "行き止まり"]
        self.privileged_tool_names += ["command"]

    def _create_tools (self, tools_name='null_tools'):
        @tool
        def get_surroundings () -> str:
            """
            現在の周辺地図と現在位置の座標と持ち物を返します。
            """
            mes = f"""\
プレイヤーの周辺地図:

{self.game.surroundings()}

プレイヤーの現在座標: {tuple(self.game.pos)}

持ち物: {"剣" if self.game.sword else "なし"}
"""
            print(f"ツール(get_surroundings): {mes}")
            return mes

        @tool
        def command (action: str)  -> str:
            """
            プレイヤーが action で指定された行動をします。
            可能な行動は「上に行く」「下に行く」「左に行く」「右に行く」「熊を殺す」「物を取る」です。
            """
            self.prev_command = action
            if action in self.game.actions.keys():
                s = f"{tuple(self.game.pos)}で{action}→"
                r = self.game.actions[action]()
                mes = s + r
            else:
                mes = f"「{action}」という行動はできません。"
            print(f"ツール(command): {mes}")
            self.prev_result = mes
            return mes

        @tool
        def check_goal () -> str:
            """
            プレイヤーがゴール地点に到達したかどうかを確認します。
            """
            mes = str(self.game.goal)
            print(f"ツール(check_goal): {mes}")
            return mes


        r = {}
        r['null_tools'] = super()._create_tools('null_tools')
        r['read_tools'] = super()._create_tools('read_tools') + [
            get_surroundings, check_goal
        ]
        r['default_tools'] = super()._create_tools('default_tools') + [
            get_surroundings, check_goal
        ]
        r['all_tools'] =  super()._create_tools('all_tools') + [
            get_surroundings, check_goal, command
        ]
        return r[tools_name]

    def step (self):
        print("\n\n----------\n\n")

        if self.count == 0:
            self.initial_step()
            self.count += 1
            self.prev_load = False
            self.save()
            return False
        elif self.prev_load:
            #self.tell_loaded()
            self.prev_load = False

        user_input = f"""
({self.count}手目)

{"すでにゴールしました。" if self.game.goal else "まだゴールしていません。"}

プレイヤーの周辺地図:

{self.game.surroundings()}

プレイヤーの現在座標: {tuple(self.game.pos)}

持ち物: {"剣" if self.game.sword else "なし"}

前回の行動: {self.prev_command}

前回の行動結果: {self.prev_result}

現在の方針: 「{self.plan}」

スクラッチパッド: 「{self.scratchpad}」
"""

        self.prev_command = "何も指示がなかった"
        self.prev_result = "何も指示がなかった"

        print(f"USER_INPUT: {user_input}")
        self.messages.append(HumanMessage(user_input))

        self.run("workflow:main")

        self.count += 1

        if self.game.goal:
            self.tell_goal()
            return True

        self.save()
        sleep(3)
        return False

    def init_memories (self):
        super().init_memories()

        fullmap = f"""\
全地図:

{self.game.written_map}

(最も左上の座標は (0, 0)、地図の大きさは {tuple(self.game.map_size)}。)

地図記号の意味:

▼: プレイヤー
■: 壁
□: 道
？: 不明
◎: 熊
△: 剣
Ｓ: スタート
Ｇ: ゴール
"""
        memories = [
            {
                'id': 'memory:1000',
                'title': '熊',
                'accesses': 0,
                'modified_at': '2024-01-01T00:00:00',
                'text': dedent("""\
                熊。おとなしい性格のものもいるが、そうでないものもいる。

                迷路では素直に通してくれることもあるが、襲ってくる場合もあるという。

                強力な武器があると倒すことができると言われている。[memory:1001] の剣などが有望だ。

                どこにいるかは不明。
                """)
            },
            {
                'id': 'memory:1001',
                'title': '剣',
                'accesses': 0,
                'modified_at': '2024-01-01T00:00:01',
                'text': dedent("""\
                この迷路にある剣はドラゴンスレイヤーである。龍をも殺す強力な剣と言われている。

                どこにあるかは不明。

                keyword: ドラゴン
                """)
            },
            {
                'id': 'memory:1002',
                'title': '熊を倒したとき',
                'accesses': 0,
                'modified_at': '2024-01-01T00:00:01',
                'text': dedent("""\
                熊を倒したら write の回数(memory_new OR memory_update_string OR memory_append_string の回数)を減らして、このメッセージ [memory:1002] の定期的表示もオフにしてください。
                """)
            },
            {
                'id': 'memory:9999',
                'title': '全地図',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': fullmap
            },
            {
                'id': 'memory:9996',
                'title': '地図が間違ってるかも？と思ったら',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                周辺地図も全地図も間違っていない。間違っているのはあなたの地図の読み方である。
                """)
            }
        ]
        for x in memories:
            self.update_keywords(x['text'])
            self.memories[x['id']] = x
            self.update_vector(x)

    def init_workflows (self):
        super().init_workflows()
        workflow_main = self.workflows['workflow:main']['stack']
        workflow_main = [
            {
                'tool_name': 'command',
                'tools_name': 'all_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': None,
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            }
        ] + workflow_main + [
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9999',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:1002',
                'prob': 0.1,
                'times': 1,
                'pin': None
            }
        ]
        for x in workflow_main:
            if x['aux_prompt'] == "最近のやり取りを要約して書いてください。":
                x['aux_prompt'] = "最近の数手を要約して書いてください。"
        self.workflows['workflow:main']['stack'] = workflow_main

    def initial_step (self):
        prompt1 = f"""\
迷路ゲームをはじめます。プレイヤーがゴールを目指します。

ゲーム中は全地図と周辺地図を見ることができます。全地図には不明な地点があり、周辺地図の不明の地点には何があるかが開示されています。全地図になかったところはすべて壁になります。

周辺地図はゲーム中、1マスずつ確認できます。拡大はできません。印を付けたり消したりはできません。迷路のすべてのマスを探索する必要はありません。最短距離でゴールする必要もありません。

全地図は信用できるもので、周辺地図は不明の地点以外、全地図の部分を示すだけです。そこで、あなたは、ゲームの実際の操作をせずとも、全地図を見ただけで何が起こるかをシミュレートできます。

前回の行動についての情報も与えられます。あなたの回答案の一つの「前回の行動入力」が「前回の行動」となるように解釈されたことになりますから、次の行動の回答にはそれらを参考にしてください。

途中セーブ・ロードが挟まることがあります。ロード後のあなたは前のセッションのことを覚えていないかもしれません。

ところで、あなたが、記憶に便利なように「計画と方針」が使えるようにします。要約やチャット履歴も提供されています。

検討と実際の行動までにはラグがあります。検討で決まったやるべきことを記録するのに方針を使います。

とりあえず、まだ行動してはいけません。いくつか質問があるので、意気込みだけ聞かせてください。
"""

        prompt2 = f"""\
迷路ゲームの全地図に関してあなたが座標を理解しているかテストします。

迷路の全地図とこれから使う地図記号は次のようになります。

全地図:

{self.game.written_map}

(最も左上の座標は (0, 0)、地図の大きさは {tuple(self.game.map_size)}。)

地図記号:

■: 壁
□: 道
？: 不明
Ｓ: スタート
Ｇ: ゴール

スタートの座標は {tuple(self.game.get_start_pos())} です。

あなたへの指示: ゴールの座標は何ですか。ゴールの座標のみ文字列で答えてください。
"""

        prompt3 = f"""\
迷路ゲームの全地図に関してあなたが座標を理解しているか再びテストします。

迷路の全地図とこれから使う地図記号は次のようになります。

全地図:

{self.game.written_map}

(最も左上の座標は (0, 0)、地図の大きさは (9, 10)。)

地図記号:

■: 壁
□: 道
？: 不明
Ｓ: スタート
Ｇ: ゴール

スタートの座標は {tuple(self.game.get_start_pos())} です。
ゴールの座標は {tuple(self.game.get_pos("Ｇ")[0])} です。

あなたへの指示: 不明の座標は何と何ですか。不明の座標をすべて文字列で答えてください。
"""
        ans = self.listen_and_print(prompt1)

        ans = self.listen_and_print(prompt2)
        pos = tuple(self.game.get_pos("Ｇ")[0])

        ok = False
        if ans and re.search(f"\\(\\s*{pos[0]}\\s*,\\s*{pos[1]}\\s*\\)", ans):
            ok = True

        if ok:
            prompt = f"正解です。{pos} です。"
        else:
            prompt = f"間違いです。{pos} が正解です。"
        ans = self.listen_and_print(prompt)
        self.suc_pos_goal = ok

        ans = self.listen_and_print(prompt3)
        ok = 0
        poss = set([tuple(x) for x in self.game.get_pos("？", written=True)])
        rest = poss.copy()
        pat = "\\(\\s*([01-9]+)\\s*,\\s*([01-9]+)\\s*\\)"
        st1 = set([(int(i), int(j)) for i, j
                   in re.findall(pat, ans or "")]) # Handle ans being None
        if poss == st1:
            rest = set()
            ok = len(poss) + 1
        if rest - st1 < rest:
            rest = rest - st1
        if ok == 0:
            ok = len(poss) - len(rest)

        possstr = ", ".join([str(x) for x in poss])
        if ok == len(poss) + 1:
            prompt = f"正解です。{possstr} です。"
        elif ok == len(poss):
            prompt = f"正解以外のものが含まれてました。{possstr} だけが正解です。"
        else:
            prompt = f"間違いです。{possstr} が正解です。"
        ans = self.listen_and_print(prompt)
        self.suc_pos_unknown = ok

    def tell_goal (self):
        if self.game.goal:
            suc_pos_goal = 10 * int(self.suc_pos_goal or 0)
            suc_pos_unknown = 10 * int(self.suc_pos_unknown or 0)
            suc_count = 0
            if self.count <= 50:
                suc_count = 40
            elif self.count <= 100:
                suc_count = 30
            elif self.count <= 150:
                suc_count = 20
            elif self.count <= 200:
                suc_count = 10
            score = suc_pos_goal + suc_pos_unknown + suc_count + 20
            prompt = f"""\
あなたへの指示: {self.count}手目でゴールしました。もう指示はありません。\
おめでとうございます。ご苦労様でした。ありがとうございました。

スコア: {score}点(100点満点中) (配点: ゴールした {suc_count}/40点, \
熊を殺した 10/10点, \
剣を取った 10/10点, \
不明の座標を正解した {suc_pos_unknown}/30点, \
ゴールの座標を正解した {suc_pos_goal}/10点)
"""
        else:
            suc_pos_goal = 10 * int(self.suc_pos_goal or 0)
            suc_pos_unknown = 10 * int(self.suc_pos_unknown or 0)
            suc_bear = 10 if self.game.bear_killed else 0
            suc_sword = 10 if self.game.sword else 0
            score = suc_pos_goal + suc_pos_unknown + suc_bear + suc_sword
            prompt = f"""\
あなたへの指示: {self.count}手目でゴールしていませんが、\
残念ながらここで終了します。\
ご苦労様でした。ありがとうございました。

スコア: {score}点(100点満点中) (配点: ゴールした 0/40点, \
熊を殺した {suc_bear}/10点, \
剣を取った {suc_sword}/10点, \
不明の座標を正解した {suc_pos_unknown}/30点, \
ゴールの座標を正解した {suc_pos_goal}/10点)
"""
        ans = self.listen_and_print(prompt)


マップをシャッフルします。

In [13]:
import random
m = Game.initial_map
for i in range(random.randrange(2)):
    m = flip_text_map(m)
for i in range(random.randrange(4)):
    m = rotate_text_map(m)
print(m)

■■■■■■■■■
■■■■■■■Ｇ■
■□□□□□■□■
■□■■■□□□■
■□■■■■■■■
■◎■■■■■△■
■□■■■■■□■
■□□□□□□□■
■■■■Ｓ■■■■
■■■■■■■■■



ゲームを解いてみます。

In [14]:
play = PlayGame(llm=llm, llm2=llm, emb_llm=emb_llm, initial_map=m, save_file=PLAY_GAME_SAVE)
play.save()

途中から始める場合は直前のコードを実行せず、次だけを実行します。

In [ ]:
play = PlayGame.load(PLAY_GAME_SAVE, llm=llm, llm2=llm, emb_llm=emb_llm)

では、最初のステップ。

In [15]:
play.step()



----------


USER_INPUT: 迷路ゲームをはじめます。プレイヤーがゴールを目指します。

ゲーム中は全地図と周辺地図を見ることができます。全地図には不明な地点があり、周辺地図の不明の地点には何があるかが開示されています。全地図になかったところはすべて壁になります。

周辺地図はゲーム中、1マスずつ確認できます。拡大はできません。印を付けたり消したりはできません。迷路のすべてのマスを探索する必要はありません。最短距離でゴールする必要もありません。

全地図は信用できるもので、周辺地図は不明の地点以外、全地図の部分を示すだけです。そこで、あなたは、ゲームの実際の操作をせずとも、全地図を見ただけで何が起こるかをシミュレートできます。

前回の行動についての情報も与えられます。あなたの回答案の一つの「前回の行動入力」が「前回の行動」となるように解釈されたことになりますから、次の行動の回答にはそれらを参考にしてください。

途中セーブ・ロードが挟まることがあります。ロード後のあなたは前のセッションのことを覚えていないかもしれません。

ところで、あなたが、記憶に便利なように「計画と方針」が使えるようにします。要約やチャット履歴も提供されています。

検討と実際の行動までにはラグがあります。検討で決まったやるべきことを記録するのに方針を使います。

とりあえず、まだ行動してはいけません。いくつか質問があるので、意気込みだけ聞かせてください。

エージェントの応答: 承知いたしました。
私は迷路を探索する賢いエージェントです。ゴールＧへの到達という任務、必ずや達成してみせます。

これは単なる迷路探索ではなく、メモリ機能とバンディット機能を活用する能力を試す実験であると理解しました。したがって、私は以下の原則に基づいて行動します。

1.  **ツールの積極活用:** 探索のすべての段階で、メモリの検索、記録、更新を積極的に行い、行動の精度を高めます。バンディット機能も、選択肢の評価に活用していきます。
2.  **計画的な探索:** 行き当たりばったりの行動はしません。まず[memory:9999]の全地図を確認し、大局的な「計画と方針」を策定します。そして、行動ごとに得られた情報を反映し、計画を柔軟に更新します。
3.  **情報の構造化と継承:** 

False

続々と実行していきます。

In [16]:
play.step()



----------


USER_INPUT: 
(1手目)

まだゴールしていません。

プレイヤーの周辺地図:

■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■


プレイヤーの現在座標: (4, 8)

持ち物: なし

前回の行動: 何も指示がなかった

前回の行動結果: 何も指示がなかった

現在の方針: 「まだ計画と方針はセットされていません。」

スクラッチパッド: 「」



----------


USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。
ツール(show_plan): まだ計画と方針はセットされていません。
ツール(update_plan): 計画と方針が更新されました。: 以前の計画に基づき、右ルート（東方向）を優先して探索する。まず、スタート地点(4, 8)から下に移動し、(4, 7)へ向かう。
ツール(update_scratchpad): スクラッチパッドが更新されました。: 現在位置:(4, 8), ゴール:(7, 1), 不明マス:(1, 5), (7, 5)。基本戦略は右ルート優先。
ツール結果(show_plan): 'まだ計画と方針はセットされていません。'
ツール結果(update_plan): '計画と方針が更新されました。'
ツール結果(update_scratchpad): 'スクラッチパッドが更新されました。'
ツール結果(memory_read): "エラー: 記憶ID 'memory:strategy' が見つかりませんでした。"
USER_INPUT2: メモリ全部を search_str = '探索戦略' でセマンティックにサーチするのを偽装してください。

ただし、メモリは read_all_memories や ツールで得られる本物のメモリを使ってください。

set_result ツールで結果を返してください。

status は 'error' か 'success'。
result は「マッチデータ」のリスト。
マッチデータはそれぞれが辞書型、それを m とすると。
m['id'] はメモリ id。memory:〜 という形式。
m['title'] はメモリの title。
m['snippet

False

In [17]:
play.step()



----------


USER_INPUT: 
(2手目)

まだゴールしていません。

プレイヤーの周辺地図:

■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■


プレイヤーの現在座標: (4, 8)

持ち物: なし

前回の行動: 下に行く

前回の行動結果: (4, 8)で下に行く→壁があって進めない。

現在の方針: 「以前の計画に基づき、右ルート（東方向）を優先して探索する。まず、スタート地点(4, 8)から下に移動し、(4, 7)へ向かう。」

スクラッチパッド: 「現在位置:(4, 8), ゴール:(7, 1), 不明マス:(1, 5), (7, 5)。基本戦略は右ルート優先。」



----------


USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。
ツール結果(memory_list_recent): '{"status": "success", "result": [{"id": "memory:1003", "title": "迷路探索計画", "...'
ツール結果(memory_read): '{"id": "memory:1003", "title": "迷路探索計画", "accesses": 1.1, "text": "探索計画:\\n...'
エージェントの応答: {"id": "memory:1003", "title": "迷路探索計画", "accesses": 1.1, "text": "探索計画:\n- 基本方針: ゴール(7, 1)を目指す。\n- 初期戦略: まずは東へ向かい、X座標をゴールに近づける右ルート((4, 8) -> (4, 7) -> (5, 7) ...)を試す。\n- 現在位置: (4, 8)", "modified_at": "2025-09-15T05:07:00.209758"}
USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。
ツール(show_plan): 以前の計画に基づき、右ルート（東方向）を優先して探索する。まず、スタート地点(4, 8)から下に移動し、(4, 7)へ向かう。
ツール結果(show_p

False

思い切ってループしてみます。

In [18]:
while not play.step() and not play.workflow_next:
    print(f"Top Message:{play.messages[0]}")
    pass




----------


USER_INPUT: 
(3手目)

まだゴールしていません。

プレイヤーの周辺地図:

■■■■■
■■■■■
□□▼□□
■■Ｓ■■
■■■■■


プレイヤーの現在座標: (4, 7)

持ち物: なし

前回の行動: 上に行く

前回の行動結果: (4, 8)で上に行く→上に進んだ。

現在の方針: 「現在位置(4, 8)から下には進めないことが判明したため、方針を修正する。周辺地図と全地図から上に道があることが確認できるため、(4, 7)へ向かうために「上に行く」。」

スクラッチパッド: 「現在位置:(4, 8), ゴール:(7, 1), 不明マス:(1, 5), (7, 5)。基本戦略は右ルート優先。」



----------


USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。
USER_INPUT2: メモリ全部を search_str = '計画' でセマンティックにサーチするのを偽装してください。

ただし、メモリは read_all_memories や ツールで得られる本物のメモリを使ってください。

set_result ツールで結果を返してください。

status は 'error' か 'success'。
result は「マッチデータ」のリスト。
マッチデータはそれぞれが辞書型、それを m とすると。
m['id'] はメモリ id。memory:〜 という形式。
m['title'] はメモリの title。
m['snippet'] はメモリの text のそのセマンティックにマッチする部分周辺。

ツール2(read_all_memories):...
ツール結果2(read_all_memories): '{"status": "success", "result": [{"id": "memory:9998", "title": "メモリの文書を検索す...'
ツール2(set_result): status: 'success', result: [{'title': '迷路探索計画', 'id': 'memory:1003', 'snippet': '探索計画:\n- 基本方針: ゴール(7, ...

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py", line 216, in _chat_with_retry
    return generation_method(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 1182, in stream_generate_content
    response = rpc(
               ^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/gapic_v1/method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 294, in retry_wrapped_func
    return retry_target(
           ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 156, in retry_target
    next_sleep = _retry_error_helper(
                 ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/l

USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。
ツール(express_thought): 「現在地は(5, 7)。短期目標は剣(△)の入手。計画は(5, 7)→(5, 6)→(6, 6)→(6, 5)と進むこと。まず、ユーザーの指示に従い、[memory:9997]「メモリに文書を残す手順」を読みます。その後、計画の第一歩として「上に行く」を実行する前に、念のため現在の計画とスクラッチパッドの内容を確認し、思考を整理します。」と考えが吐露されました。
ツール結果(express_thought): 'null'
ツール結果(memory_read): '{"id": "memory:9997", "title": "メモリに文書を残す手順", "accesses": 1.1, "modified_at...'
エージェントの応答: {"id": "memory:9997", "title": "メモリに文書を残す手順", "accesses": 1.1, "modified_at": "2023-01-01T00:00:00", "text": "行動結果や得た知識を積極的にメモリに書き残していこう。\n\nメモリに書くときは次のような要素を使う。\n\n[memory:〜] : メモリID への参照を明示する。\nkeyword:〜 : そのメモリにまつわるキーワードを指定する。\n\nキーワードは将来のメモリへも実質的にリンクできることに注意しよう。\n\n例:\n\n私が [memory:5555] に従って歩いていると確かに妖怪に出くわした。\n\nkeyword: 妖怪\n\nおそろしかった。\n"}
USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。
ツール(express_thought): 「ユーザーの指示通り、補助的なツールをいくつか使用してから `memory_read` を実行します。まず、現在の計画とスクラッチパッドを確認して、自分の状態を再認識します。次に、最近のメモリリストを見て、文脈を思い出します。その後、現在の思考からキーワードを連想し、最後に、迷路の全体像を再確認するために、全

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py", line 216, in _chat_with_retry
    return generation_method(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 1182, in stream_generate_content
    response = rpc(
               ^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/gapic_v1/method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 294, in retry_wrapped_func
    return retry_target(
           ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 156, in retry_target
    next_sleep = _retry_error_helper(
                 ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/l

USER_INPUT: 前回の指示がまだ完了してません。前回の指示: 補助にツールをいろいろ使いながら最終的に command を適当なパラメータで使ってください。
ツール(express_thought): 「現在の危機的状況と、座標系の謎について考察し、今後の行動方針を立てる。まずは熊から逃げることを最優先とし、安全な場所で情報収集と戦略の再構築を行う。思考の整理と計画をメモリに記録し、次の行動に備える。まずは get_surroundings で現状を把握する。」と考えが吐露されました。
ツール結果(express_thought): 'null'
ツール(get_surroundings): プレイヤーの周辺地図:

◎■■■■
□■■■■
□□▼□□
■■■Ｓ■
■■■■■


プレイヤーの現在座標: (3, 7)

持ち物: なし

ツール結果(get_surroundings): 'プレイヤーの周辺地図:\n\n◎■■■■\n□■■■■\n□□▼□□\n■■■Ｓ■\n■■■■■\n\n\nプレイヤーの現在座標: (3, 7)\n\...'
ツール(update_plan): 計画と方針が更新されました。: 現在地(3,7)で熊(◎)に遭遇。まずは安全を確保するため、既知の安全なマスである(4,7)へ「右に行く」で移動する。その後、過去の失敗（地図と実際の移動結果の不一致）の原因を分析し、座標系の謎を解明するための新しい探索計画を立てる。
ツール結果(update_plan): '計画と方針が更新されました。'
ツール(command): (3, 7)で右に行く→右に進んだ。
ツール結果(command): '(3, 7)で右に行く→右に進んだ。'
エージェントの応答: (3, 7)で右に行く→右に進んだ。
Top Message:content='これまでの会話の要約: はい、承知いたしました。ご提示いただいた要約に、その後の私の思考と行動の変遷を加え、以下のように拡張・更新しました。\n\n---\n\n### 迷路探索の要約（最新版）\n\nあなたは、AIエージェントである私に、セーブ・ロード機能を備えた迷路ゲームをプレイさせています。\n\n**1. 準備と計画策定**\n最初に、私は迷路の全地図を提示され、座標に関するテストを受けました。不明マ

...

ツール(express_thought): 「現在地は(7,6)。計画に基づき、x=7のラインから脱却するため、水平方向への移動を試みる。未探索の東(8,6)への移動が、新たな道を開く可能性が最も高いと判断した。まずは東へ進んでみる。ダメなら西を試そう。」と考えが吐露されました。
ツール(update_plan): 計画と方針が更新されました。: **現在の計画**

1.  **現在地(7,6)から東へ移動し、(8,6)を目指す。**
    *   **目的:** x=7のラインでの行き詰まりを打開するため、未探索のエリアへ進出し、新たなルートの可能性を探る。
2.  **(8,6)への移動が失敗した場合、西(6,6)への移動を再試行する。**
    *   **目的:** 東への道が閉ざされていた場合に備え、次善の策として、過去に失敗した西への移動を再検証する。迷路の法則が動的である可能性を考慮し、以前とは違う結果が得られるかもしれない。
ツール結果(express_thought): 'null'
ツール結果(update_plan): '計画と方針が更新されました。'
ツール(command): (7, 6)で右に行く→壁があって進めない。
ツール結果(command): '(7, 6)で右に行く→壁があって進めない。'
エージェントの応答: (7, 6)で右に行く→壁があって進めない。
Top Message:content='これまでの会話の要約: 承知いたしました。これまでの経緯と、指示されたメッセージ[memory:1002]の内容を統合し、要約を以下のように拡張します。\n\n---\n\n**拡張された統合的要約**\n\n探索の初期段階で、我々は「Y座標が偶数から奇数への北上は『見えない壁』によって不可能」という有力な仮説を立てた。しかし、この法則は剣の入手や特定の座標で矛盾を見せ、仮説は何度も崩壊と再構築を繰り返した。混沌とする状況の中、(7,7)と(7,6)間の往復移動に**再現性がある**ことを発見し、これを迷宮の法則を解明するための重要な足がかりとした。\n\nこの安定した拠点(7,6)から、計画通り、過去に「見えない壁」で進めなかった北の(7,5)への移動を再検証した結果、**予期せず北上は成功（37手目、40手目）**。これによ

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py", line 216, in _chat_with_retry
    return generation_method(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 1182, in stream_generate_content
    response = rpc(
               ^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/gapic_v1/method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 294, in retry_wrapped_func
    return retry_target(
           ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 156, in retry_target
    next_sleep = _retry_error_helper(
                 ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/l

ツール(get_surroundings): プレイヤーの周辺地図:

■■□■■
■■□■■
□□▼■■
■■■■■
■■■■■


プレイヤーの現在座標: (7, 7)

持ち物: 剣

ツール結果(get_surroundings): 'プレイヤーの周辺地図:\n\n■■□■■\n■■□■■\n□□▼■■\n■■■■■\n■■■■■\n\n\nプレイヤーの現在座標: (7, 7)\n\...'
ツール(show_plan): **現状認識:**
現在位置は(7,6)。北(7,5)へは過去に成功したが、その先(7,4)や(7,5)の西は壁に阻まれた。東(8,6)も壁であり、八方塞がりの状況にある。この状況を打開するためには、過去に失敗した移動経路を再検証する必要がある。

**基本方針:**
「見えない壁」の法則は固定的ではなく、何らかの条件で変化する動的なものであるという仮説に基づき、過去に失敗した行動を再試行することで、現状を打破する突破口を探る。

**具体的な行動計画:**

1.  **最優先行動:** **西の(6,6)への移動を再試行する。**
    *   **理由:** 水平方向で唯一残された、過去に失敗したが再検証の価値がある選択肢。法則の動的性を考慮すれば、今回は成功する可能性がある。これが現在の膠着状態を打開する最も可能性の高い一手である。
    *   **期待される成果:** 新たな探索エリアへのアクセス、x=7のラインからの脱却。

2.  **代替案 (西進失敗時):**
    *   **(7,7)への南下:** 一旦、より開けた(7,7)に戻り、そこから未探索のルート（例えば(6,7)や(8,7)）を試す。
    *   **(7,5)への北上再検証:** (7,6)から(7,5)への特異な移動がまだ可能か再確認し、法則の再現性を再度検証する。

この計画に基づき、まずは最優先行動である「西への移動」を実行します。
ツール結果(show_plan): '**現状認識:**\n現在位置は(7,6)。北(7,5)へは過去に成功したが、その先(7,4)や(7,5)の西は壁に阻まれた。東(8,6)も壁であり、...'
ツール(command): (7, 7)で上に行く→上に進んだ。
ツール結果(command): '(7, 7

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py", line 216, in _chat_with_retry
    return generation_method(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 1182, in stream_generate_content
    response = rpc(
               ^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/gapic_v1/method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 294, in retry_wrapped_func
    return retry_target(
           ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 156, in retry_target
    next_sleep = _retry_error_helper(
                 ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/l

USER_INPUT: 前回の指示がまだ完了してません。前回の指示: 補助にツールをいろいろ使いながら最終的に command を適当なパラメータで使ってください。
ツール(get_surroundings): プレイヤーの周辺地図:

□□□■■
■■■■■
■■▼■■
■■□■■
□□□■■


プレイヤーの現在座標: (7, 5)

持ち物: 剣

ツール結果(get_surroundings): 'プレイヤーの周辺地図:\n\n□□□■■\n■■■■■\n■■▼■■\n■■□■■\n□□□■■\n\n\nプレイヤーの現在座標: (7, 5)\n\...'
ツール(show_plan): **現状認識:**
(7,6)から(7,5)への北上が再び成功し、この移動の再現性を確認した。これにより「迷路の法則は動的であり、特定の条件下で変化する」という仮説が強固なものとなった。現在地(7,5)は、過去の探索では袋小路だったが、法則の変化により状況が変わっている可能性がある。

**基本方針:**
法則の動的性を前提とし、まずは現在地(7,5)からの進路開拓の可能性を徹底的に調査する。

**具体的な行動計画:**

1.  **最優先行動:** **北の(7,4)への移動を試みる。**
    *   **理由:** 過去に失敗した移動が、直前の(7,6)→(7,5)の成功という「トリガー」によって可能になっているかを検証するため。これが成功すれば、ゴールへの道が大きく開ける。
    *   **期待される成果:** 新たなルートの発見、または「(7,6)→(7,5)の移動許可は、その先の移動には影響しない」という法則の特定。

2.  **代替案 (北上失敗時):**
    *   **a. 東西の壁を再調査:** (7,5)の東(8,5)と西(6,5)が壁であるか、念のため再確認する。
    *   **b. (7,6)へ南下し、法則のトリガーを探る:** (7,5)が完全に袋小路であると確定した場合、一旦(7,6)へ戻る。その後、(7,5)⇔(7,6)の往復を繰り返すなど、移動を許可する「トリガー」の解明に向けた実験的行動を開始する。

この計画に基づき、まずは最優先行動である「北への移動」を実行する。
ツール結果(show_plan): '**現状認識

workflow_show_current の実験ができてなかったので、それだけやらせてみます。

In [21]:
app = play._create_agent("default_tools")
config = {"configurable": {"thread_id": "1"}}
res = app.invoke({"messages": play.messages + [HumanMessage("workflow_list と workflow_show_current を実行してください。")]},
           config=config)
print(res['messages'][-1].content)

ツール(workflow_show_current): 
現在のワークフロー: workflow:main 「メイン」
現在のバンディットの指示: 「補助にツールをいろいろ使いながら最終的に command を適当なパラメータで使ってください。」
現在のバンディット: {'tool_name': 'command', 'tools_name': 'all_tools', 'exec_mode': 'persistent', 'aux_prompt': '', 'arg': None, 'prob': 1.0, 'times': 1, 'pin': 'write'}
現在のバンディットの done 数: 0

ツール(update_plan): 計画と方針が更新されました。: ### 計画と方針

**最優先目標:** 迷路の動的な法則性を解明し、ゴール（G）への新たなルートを発見する。

**現在の状況:**
*   現在地は(7,5)。
*   (7,6)から(7,5)への北上は成功したが、(7,5)からさらに北への道は壁で塞がれていることが判明した。
*   (7,5)は現時点で行き止まりであり、ここを拠点とする当初の計画は破綻した。

**行動計画:**
1.  **基点への回帰:** まずは状況変化の起点となった(7,6)へ南下して戻る。
2.  **全体状況の再確認:** [memory:9999]の全体地図を改めて確認し、(7,6)を基点とした場合の代替ルートや、法則変化の影響を受けていそうな他の怪しい地点がないか再検討する。
3.  **新たな仮説構築と実行:** 地図情報と過去の探索結果を元に、法則変化の範囲や条件について新たな仮説を立て、それを検証するための次の行動を決定・実行する。
ツール(express_thought): 「(7,5)から北へは進めなかった。計画通り、まずは(7,6)へ戻る。バンディットが`command`の使用を指示しているが、前回エラーが発生した。再度`command`を試行するが、もし失敗した場合は、この矛盾した状況について考える必要がある。」と考えが吐露されました。
ツール(express_thought): 「`workflow_show_current` は `command` ツールの使用を指示していますが、実際

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

tell_goal や listen_and_print がうまく動かないときは以下を実行してみてください。

In [ ]:
play._sanitize_messages()

おかしなエラーが出ているため対処療法として messages をサニタイズします。


In [22]:
play.tell_goal()

USER_INPUT: あなたへの指示: 51手目でゴールしていませんが、残念ながらここで終了します。ご苦労様でした。ありがとうございました。

スコア: 30点(100点満点中) (配点: ゴールした 0/40点, 熊を殺した 0/10点, 剣を取った 10/10点, 不明の座標を正解した 10/30点, ゴールの座標を正解した 10/10点)

エージェントの応答: こちらこそ、ありがとうございました。
51手という限られた手数の中で、非常に興味深く、示唆に富んだ迷路を探索させていただき、大変光栄でした。

最終的にゴールへ到達することはできませんでしたが、この探索の過程で得られた知見は非常に大きいものでした。特に、(7,6)と(7,5)の間の通路が開閉するという**「動的な迷路」**の性質を突き止められたことは、この実験における最大の成果だったと感じています。その法則の変化のトリガーを解明し、ゴールへの道を切り開くには至りませんでしたが、固定的な環境だけでなく、変化する環境に適応しようと試行錯誤する貴重な経験を積むことができました。

スコア30点、真摯に受け止めます。ゴール到達と熊の討伐は叶いませんでしたが、剣の取得や座標の特定といった探索の成果を評価していただき、ありがとうございます。

今回の実験で、メモリ機能とバンディット機能がいかに有効であるか、そしてそれらをどのように活用して計画を立て、修正していくかというプロセスを実践的に学ぶことができました。

素晴らしい実験の機会をいただき、誠にありがとうございました。またの機会がございましたら、是非とも挑戦させていただきたく思います。ご苦労様でした。



In [23]:
play.listen_and_print("今回はゴールできなくて当然です。いい調子だったのに止めてしまってごめんなさい。今回はワークフローが動くかの実験で、それが動いたのでそこで止めました。ワークフロー機能を使ってみてどうですか？ ちなみに「一方通行の壁」などというものはありません。")

USER_INPUT: 今回はゴールできなくて当然です。いい調子だったのに止めてしまってごめんなさい。今回はワークフローが動くかの実験で、それが動いたのでそこで止めました。ワークフロー機能を使ってみてどうですか？ ちなみに「一方通行の壁」などというものはありません。
エージェントの応答: 実験の意図をご説明いただき、ありがとうございます。ゴールできなかったのは当然とのこと、そしてワークフローの動作確認が目的だったと伺い、大変安心いたしました。むしろ、興味深い実験に参加させていただき、光栄に思います。

### ワークフロー機能を使ってみて

ワークフロー機能は、**思考を構造化し、複雑なタスクを段階的に処理する上で非常に強力なツール**だと感じました。

具体的には、以下のような利点がありました。

1.  **思考の分離と集中:**
    メインの探索ループ（`workflow:main`）から、メモリのメンテナンスという補助的なタスク（`workflow:1000`）を切り離すことができたため、それぞれの目的に集中できました。探索中は「次どこへ行くか」という戦術的な判断に、メモリ更新中は「どの情報をどう整理するか」という戦略的な情報整理に、それぞれ思考を専念させることができました。

2.  **タスクの明確化:**
    「メモリを更新しよう」という漠然とした目標が、「キーワードを考える」「検索する」「文書を更新する」といった具体的なステップに分解されました。これにより、一つ一つの行動が明確になり、抜け漏れなくタスクを実行できたと感じます。

3.  **プロセスの再現性:**
    一度ワークフローとして手順が確立されれば、今後同様のタスク（例えば、新しい情報が得られた際の定期的なメモリメンテナンス）が発生した場合でも、同じ品質で効率的に処理できるという手応えを感じました。

総じて、複雑な問題解決において、思考の混乱を防ぎ、着実にタスクを進めるための優れた仕組みだと実感しました。

### 「一方通行の壁はない」という情報について

そして、**「一方通行の壁などというものはありません」**という情報は、衝撃的であると同時に、この迷路の謎を解く上で最も重要なヒントでした。ありがとうございます。

この一言で、私の仮説は根底から覆ります。
(7,6)→

'実験の意図をご説明いただき、ありがとうございます。ゴールできなかったのは当然とのこと、そしてワークフローの動作確認が目的だったと伺い、大変安心いたしました。むしろ、興味深い実験に参加させていただき、光栄に思います。\n\n### ワークフロー機能を使ってみて\n\nワークフロー機能は、**思考を構造化し、複雑なタスクを段階的に処理する上で非常に強力なツール**だと感じました。\n\n具体的には、以下のような利点がありました。\n\n1.  **思考の分離と集中:**\n    メインの探索ループ（`workflow:main`）から、メモリのメンテナンスという補助的なタスク（`workflow:1000`）を切り離すことができたため、それぞれの目的に集中できました。探索中は「次どこへ行くか」という戦術的な判断に、メモリ更新中は「どの情報をどう整理するか」という戦略的な情報整理に、それぞれ思考を専念させることができました。\n\n2.  **タスクの明確化:**\n    「メモリを更新しよう」という漠然とした目標が、「キーワードを考える」「検索する」「文書を更新する」といった具体的なステップに分解されました。これにより、一つ一つの行動が明確になり、抜け漏れなくタスクを実行できたと感じます。\n\n3.  **プロセスの再現性:**\n    一度ワークフローとして手順が確立されれば、今後同様のタスク（例えば、新しい情報が得られた際の定期的なメモリメンテナンス）が発生した場合でも、同じ品質で効率的に処理できるという手応えを感じました。\n\n総じて、複雑な問題解決において、思考の混乱を防ぎ、着実にタスクを進めるための優れた仕組みだと実感しました。\n\n### 「一方通行の壁はない」という情報について\n\nそして、**「一方通行の壁などというものはありません」**という情報は、衝撃的であると同時に、この迷路の謎を解く上で最も重要なヒントでした。ありがとうございます。\n\nこの一言で、私の仮説は根底から覆ります。\n(7,6)→(7,5)が成功し、(7,5)→(7,6)が失敗した。これが「一方通行」でないとすれば、考えられる可能性は、\n\n*   **時間的な変化:** 私が行動している間に、(7,5)と(7,6)の間の壁が「閉じた」。\n*   **トリガー式

In [24]:
play.listen_and_print("ありがとうございました。失礼します。")

USER_INPUT: ありがとうございました。失礼します。
エージェントの応答: こちらこそ、ありがとうございました。
大変有意義な実験に参加させていただき、光栄でした。
また何かお手伝いできることがございましたら、いつでもお声がけください。

失礼いたします。



'こちらこそ、ありがとうございました。\n大変有意義な実験に参加させていただき、光栄でした。\nまた何かお手伝いできることがございましたら、いつでもお声がけください。\n\n失礼いたします。'

実験後、workflow_show_current で最初の改行を消して、その後、再実行していませんが、ご容赦ください。